## GEOSeMOSYS - exploring linear programming geospatial energy modelling 

Nandi Moksnes, Mark Howells, Will Usher

This Jupyter Notebook is dectription of how to reproduce the results in the paper:
For any questions please contact Nandi Moksnes: nandi@kth.se

## 1. The files in "GIS_data" are downloaded and placed in a "temp" folder.
Approx 25 min

In [ ]:
%load_ext autoreload
%autoreload
from datetime import datetime

date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

#Make sure you are in the /src directory when you start this script
from Download_files import *
import os


download_url_data('input_data/GIS_URL.txt', 'temp')
unzip_all('input_data/GIS_URL.txt')

## 2. The files are then projected and clipped to the administrative boundaries. 
Approx 3 hours

In [ ]:
%autoreload
from datetime import datetime
date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
print(date)

import os
from Project_GIS import *
current = os.getcwd()

os.chdir(current)

main('../GIS_data')

## 3. Through QGIS make raster to point layer and save (MANUAL STEP)

masked_UMT37S_ken_ppp_2018_1km_Aggregated: SAGA raster to point in QGIS

This step is downloaded from zenodo so it is not needed for the user to execute.

Run time: Approx 1 minute


In [ ]:
%autoreload

date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

from Download_files import *
import os
#Make sure you are in the /src directory when you start this script
os.chdir(r'C:\Users\nandi\Box Sync\PhD\Paper 3-OSeMOSYS 40x40\GIS_python_build\GEOSeMOSYS_reprod\GEOSeMOSYS_Kenya\src')
print(os.getcwd())

download_url_data("input_data/zenodo.txt", "Projected_files")

## 4. The GIS layers are prepared to for a heuristic approximation for electrified settlements
Approx 15 min

In [ ]:
%autoreload
import matplotlib.pyplot as plt
from settlement_build import *

date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

pop_shp = '../Projected_files/raster_to_point_UMT37S_Kenya.shp'
Projected_files_path = '../Projected_files'

rasterize = raster_proximity(Projected_files_path)
points = raster_to_point(rasterize, pop_shp, Projected_files_path)

## 5. Approximate location of urban settlements and the electrified settlements 1kmx1km resolution
Approx 10 min. Make sure to wait so that the shapefile and csv file is saved properly.

In [ ]:
%autoreload
from elec_start import *
import os

date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

elec_actual = 0.75  # percent
pop_cutoff = 4000  # people
dist_minigrid = 5000  # meters
dist_mv = 5000 #meters
dist_lv = 2000 #meters
dist_to_trans = 5000  # meters
dist_to_sub = 5000 #meters
dist_minig = 5000 #meters
min_night_lights = 20
max_grid_dist = 10000  # meters
max_road_dist = 300  # meters
pop_cutoff2 = 36000 # people
urban_elec_ratio = 83.5  # percent
rural_elec_ratio = 71.5  # percent
pop_actual = 52570000  # peolpe
urban = 0.275  # percent
urban_cutoff = 20000
start_year = 2018
settlement = gpd.read_file("../Projected_files/settlements.shp")

settlements = pd.DataFrame(settlement, copy=True)
urbansettlements = calibrate_pop_and_urban(settlements, pop_actual, urban, urban_cutoff)
elec_current_and_future(urbansettlements, elec_actual, pop_cutoff, dist_to_trans, dist_to_sub, dist_minig, min_night_lights,
                            max_grid_dist, urban_elec_ratio, rural_elec_ratio, max_road_dist, pop_actual, pop_cutoff2, start_year, dist_mv, dist_lv)


In [ ]:
%autoreload
import geopandas as gpd
import matplotlib.pyplot as plt

date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

shp_path = (r'..\Projected_files\elec.shp')

point = gpd.read_file(shp_path)

In [ ]:
fig, ax = plt.subplots(1, 1)
point.plot(column='elec', ax=ax)
fig.suptitle('Estimated electrified popluation (in yellow) Kenya', fontsize=18)

## 6. Download Renewable.ninja data for all 378 locations
Approx 24 h due to the max dowloads per hour

In [ ]:
%autoreload
from renewable_ninja_download import *

date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)
token = 'ed519952eff7850cece8c746347fee2d068ab988'  # add your token for API from your own log in on Renewable Ninja
time_zone_offset = 3  # Kenya is UTC + 3hours to adjust for the time zone
shapefile = '../Projected_files/new_40x40points_WGSUMT37S.shp'
path = "temp"
coordinates = project_vector(shapefile)
wind, solar = csv_make(coordinates)
down = download(path, wind, solar, token)
adjust_timezone(path, time_zone_offset)


## 7. Scale down from1x1 km to 40 x 40 km electrified and unelectrified demand cells and save to un_elec/elec.csv and demand_cells.csv

elec From the demand_cell.csv file identify the cells that does not have any electrified cells in the 40x40 cell and classify them as unelectrified.

Approx. 1 h

In [ ]:

%autoreload
from datetime import datetime

date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)
#os.chdir(r'C:\Users\nandi\Box Sync\PhD\Paper 3-OSeMOSYS 40x40\GIS_python_build\GEOSeMOSYS_reprod\GEOSeMOSYS_Kenya\src')
#print(os.getcwd())

from post_elec_GIS_functions import *
shape = '../Projected_files/Final_Polygons_Kenya.shp'
gdp = '../Projected_files/masked_UMT37S_GDP_PPP_2015_WGS_84.tif'
elec_shp = '../Projected_files/elec.shp'
demandcells = os.path.join(os.getcwd(), 'run/Demand/demand_cells.csv')

join(elec_shp, gdp, shape)
elec(demandcells)


# 8. Calculate the demand per cell based on following equations (MANUAL STEP in Excel)

![title](input_data/eq1.png)
![title](input_data/eq2.jpg)

(In Excel)
The reference scenario follows the reference scenario from [Updated least cost power development plan 2017-2037](http://gak.co.ke/wp-content/uploads/2019/02/Updated-Least-Cost-Power-Development-Plan-2017-2022-min.pdf) but deducts the un-electrified scenario which is multiplied with the [Tier 2](http://gak.co.ke/wp-content/uploads/2019/02/Updated-Least-Cost-Power-Development-Plan-2017-2022-min.pdf ) and [4 people per household](https://openknowledge.worldbank.org/bitstream/handle/10986/24368/Beyond0connect0d000technical0report.pdf?sequence=1&isAllowed=y). Full electrification is achived by 2030 following the SDG7 agenda 2030.

The vision scenario follows the vision scenario from [Updated least cost power development plan 2017-2037](http://gak.co.ke/wp-content/uploads/2019/02/Updated-Least-Cost-Power-Development-Plan-2017-2022-min.pdf) but deducts the un-electrified scenario which is multiplied with the [Tier 3](http://gak.co.ke/wp-content/uploads/2019/02/Updated-Least-Cost-Power-Development-Plan-2017-2022-min.pdf ) and [4 people per household](https://openknowledge.worldbank.org/bitstream/handle/10986/24368/Beyond0connect0d000technical0report.pdf?sequence=1&isAllowed=y). Full electrification is achieved 2022. Even though there is expected increase in population growth rate the un-electrified population was kept at the 2018 level (12.8 million un-electrified) as the growing population will be part of the overall demand from the Updated least cost power development plan, and also included in an electrified or un-electrified household.

The demand after fully electrified (2030 and 2022 respectively) is also assumed to increase with an annual growth rate of 3% and 5% respectively for all unelectrified households. This is lower than the overall expected growth (5-8% for reference and 8-11% for Vision) for the electricity demand. However, the increase in demand is also related to the industry and flagship projects which are not included in the un-electrified households and therefore a lower growth rate is assumed.



# 9. Distribution lines

The distribution lines distance is based on three parameters:
- Area of the unelectrified population
- Number of households in that area (population)
- Peak demand for households

Area of unelectrified population and peak demand
The area of the unelectrified population corresponds to the sum of 1x1km in each cell (from the estimated electrification modelling). The inhabited area is the populated unelectrified area within the 40x40 cell multiplied with a disaggregation factor which represents habitation patterns (set to 0.3 for Kenya from [Modi et al. 2005](https://lutw.org/wp-content/uploads/Energy-services-for-the-millennium-development-goals.pdf)).
To calculate the number of LV lines the max capacity needed is set to 10 kW. Following the Multi Tier framework the Tier 2 scenario peak demand corresponds to 50 W and for Tier 3 the peak demand per household is 200 W ([Bhatia & Angelou, 2015 TABLE 
ES.1](https://openknowledge.worldbank.org/bitstream/handle/10986/24368/Beyond0connect0d000technical0report.pdf?sequence=1&isAllowed=y)).

![title](input_data/eq3.png)
![title](input_data/eq4.png)
![title](input_data/eq5.png)
![title](input_data/eq6.png)
![title](input_data/eq7.png)
![title](input_data/eq8.png)

Capacity calculations are excluded as these are accounted for in the OSeMOSYS modelling. The length of the LV per cell is then multiplied with the cost per MW-km. [The cost per km is set to 4000 USD/MW-km and 15 000 USD/MW for a substation](https://iea-etsap.org/E-TechDS/PDF/E12_el-t&d_KV_Apr2014_GSOK.pdf).

## Diesel generators
The specific fuel consumption in a diesel generator is approximately 0.324 l/kWh when operating close to the rated power (approx. 70-89%) (Yamegueu et al., 2011). However, there are variations depending on the load it operates under ranging from 0.32 to 0.53 l/kWh depending on the rated power (Dufo-López et al., 2011). The Fuel Energy density is for Diesel 39.6 MJ/liter which leads to a fuel consumption of 12.672 MJ/kWh – 20.988 MJ/kWh. 1 kWh is 3.6 MJ meaning that 12.672/3.6 = 3.52 MJdiesel/MJel – 5.83 MJdiesel/MJel. 4 TJdiesel/TJel in the model. 

## 10. Build the following csv files for building the data file to OSeMOSYS
- GIS_data.csv
- capacityfactor_solar.csv
- capacityfactor_wind.csv
- capitalcost.csv
- inputactivity.csv
- outputactivity.csv
- capacitytoactivity.csv

It also generates a technology and fuel file that is convenient to paste into the base OSeMOSYS txt file for the SETs TECHNOLOGY and FUEL.

## 11. Transmission lines distance and precenses of minigrid
The cells that has at least one electrified cell is concidered to have an HV line or possibility to connect to an additional LV line and connect the remainder of the area.

Cost concidered are [2500 USD MW-km for transmission lines and 1500 USD/MW](https://iea-etsap.org/E-TechDS/PDF/E12_el-t&d_KV_Apr2014_GSOK.pdf) for substation and  [4000 USD/MW-km](https://iea-etsap.org/E-TechDS/PDF/E12_el-t&d_KV_Apr2014_GSOK.pdf) for low voltage lines.

The fixed cost of HV/LV transmission lines are set to 3.5% of the capital cost [ADB](https://www.adb.org/sites/default/files/linked-documents/46390-002-ea.pdf)

The cells that are electrified and not 5000 m from a minigrid and not 50,000 m from the exsiting HV-MV grid the cell are concidered electrified by transmission lines.

In [ ]:
%load_ext autoreload
%autoreload
from datetime import datetime
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")

print(date)

from Build_csv_files import * 

renewable_path = 'temp'
pop_shp = 'new_40x40points_WGSUMT37S.shp'
unelec = 'run/un_elec.csv'
noHV = 'run/noHV_cells.csv'
HV = 'run/HV_cells.csv'
elec = 'run/elec.csv'
Projected_files_path = '../Projected_files/'
distribution_network = 'run/Demand/Distribution_network.xlsx'

capital_cost_HV = 2.5 #kUSD MW-km
substation = 1.5 #kUSD/MW
capital_cost_LV = 4 #kUSD/MW
capacitytoactivity = 31.536 #coversion MW to TJ
distribution_cost = 'Tier3_LV_length_(km)'
path = 'run/vision/'

#Solar and wind csv files
renewableninja(renewable_path)
#Location file
GIS_file()

#calculates the shortest distance to the HV line from the center of the 40x40 cells
transmission_near = "run/Demand/transmission.shp"
#transmission_near = near_dist(pop_shp, noHV, Projected_files_path)
capital_cost_transmission_distrib(distribution_network, elec, noHV, HV, unelec, transmission_near, capital_cost_HV, substation, capital_cost_LV, capacitytoactivity, distribution_cost, path)

# 12. Update the final files that are not updated by the script
- Demandprofile.csv (KPLC, 2015 halfhourly profile)
- demandprofile_rural.csv [Williams et al., 2018](https://ieeexplore.ieee.org/document/8521099/)
- emissions.csv
- capitalcost_RET_ref.csv [NREL - mid](https://atb.nrel.gov/)
- capitalcost_RET_vision.csv [NREL - low](https://atb.nrel.gov/)
- Merge fixedcost_tnd to fixed_cost
- variable_cost.csv 
- Operational_life.csv
- battery

# 13. Run the [GEOSeMOSYS code](https://github.com/KTH-dESA/GEOSeMOSYS) by Moksnes et al. (2020) through the command line
To fit the Kenya analysis the code is modified with the following additions:
- Electrified cells are treated separatley therefore an extra row is added to the functions where Solar PV, LV lines, or final electricity is calculated. They are treated as a separate cell with the extension of "_0" and "_1" for un-electrified and electrified

# 14 a) Reference scenario

In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

!python run/run_datafile_ref.py run/update run/Run2_datafile_rev.txt run/output/Run2_datafile.txt

# 14 b) Vision 2030 scenario

In [ ]:
%autoreload
date = datetime.now().strftime("%Y %m %d-%I:%M:%S_%p")
print(date)

!python run/run_datafile_vision.py run/vision run/Kenya_X_Y_data_BIG_GJ_MW_vision2030_sustainabledevelopment.txt run/output/Run1_vision_dd.txt

# 15) Check the datafile before the run
The datafile can have Nan so those needs to be deleted as they cannot be processed in GNUMath Prog.

# 16) Run the data file on a High performance cluster
- This analysis was run on the KTH PDC resource Tegner with the performance of 2 x 12 cores Intel E5-2690v3 Haswell, 512 GB RAM 24 cores and 1  x NVIDIA Quadro K420. 

In [ ]:
!sbatch run_cplex_tegner_REF.sh